# CNMF (Complex Non-negative Matrix Factorization)

## 因子分解

In [ ]:
%%shell
git clone https://github.com/tky823/audio_source_separation.git

In [ ]:
%cd "/content/audio_source_separation/egs/nmf-example/cnmf"

In [ ]:
import sys
sys.path.append("../../../src")

In [ ]:
import numpy as np
import scipy.signal as ss
import soundfile as sf
import IPython.display as ipd
import matplotlib.pyplot as plt

In [ ]:
from algorithm.nmf import ComplexEUCNMF

In [ ]:
plt.rcParams['figure.dpi'] = 200

In [ ]:
fft_size, hop_size = 1024, 256

In [ ]:
x, sr = sf.read("../../../dataset/sample-song/sample-1_electric-guitar_8000.wav")

### 目的音源

In [ ]:
ipd.Audio(x, rate=sr)

### CNMFの実行

In [ ]:
_, _, X = ss.stft(x, nperseg=fft_size, noverlap=fft_size-hop_size)

In [ ]:
n_basis = 6
regularizer, p = 1e-4, 1.2

In [ ]:
np.random.seed(111)
nmf = ComplexEUCNMF(n_basis=n_basis, regularizer=regularizer, p=p)

In [ ]:
basis, activation, phase = nmf(X, iteration=1000)

In [ ]:
plt.figure()
plt.plot(nmf.loss, color='black')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()

### 再構成信号

In [ ]:
Z = basis[:, :, np.newaxis] * activation[np.newaxis, :, :] * phase
Z = np.sum(Z, axis=1)
_, z = ss.istft(Z, nperseg=fft_size, noverlap=fft_size-hop_size)
z = z / np.abs(z).max()
ipd.Audio(z, rate=sr)

### NMF後の各信号

In [ ]:
for idx in range(n_basis):
    Z = basis[:, idx: idx+1] * activation[idx: idx+1, :] * phase[:, idx, :]
    _, z = ss.istft(Z, nperseg=fft_size, noverlap=fft_size-hop_size)
    z = z / np.abs(z).max()
    display(ipd.Audio(z, rate=sr))

In [ ]:
for idx in range(n_basis):
    estimated_spectrogram = basis[:, idx: idx + 1] * activation[idx: idx + 1, :] * phase[:, idx, :]

    estimated_power = np.abs(estimated_spectrogram)**2
    estimated_power[estimated_power < 1e-12] = 1e-12
    log_spectrogram = 10 * np.log10(estimated_power)

    plt.figure()
    plt.pcolormesh(log_spectrogram, cmap='jet')
    plt.show()